In [1]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import io
import itertools
from packaging import version
from six.moves import range

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics
import random
import utils
import cv2

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.0.0


In [3]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [4]:
data_path = '/home/hnkulkarni/nn/CS230/Project/trashnet/data/dataset-resized/dataset-resized/'
image_paths, image_labels, image_label_text, num_classes =  utils.get_image_paths_labels(data_path)

/home/hnkulkarni/nn/CS230/Project/trashnet/data/dataset-resized/dataset-resized/
Label names are ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
Label to index are {'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}


In [5]:
# Get Images
train_images = [cv2.imread(path, 3) for path in image_paths]
train_images_fgms = [utils.forground_mask(img) for img in train_images]

In [6]:
# Download the data. The data is already divided into train and test.
# The labels are integers representing classes.
# fashion_mnist = keras.datasets.fashion_mnist
# (train_images, train_labels), (test_images, test_labels) = \
#     fashion_mnist.load_data()

# # Names of the integer classes, i.e., 0 -> T-short/top, 1 -> Trouser, etc.
# class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
#     'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [7]:
print("Shape: ", train_images_fgms[0].shape)
print("Label: ", image_labels[0], "->", image_label_text[0])

Shape:  (384, 512)
Label:  2 -> metal


In [8]:
x_train, input_shape, num_train_samples = utils.reshape(train_images_fgms)
print("x_train shape: {}".format(x_train.shape))
print("input_shape {}".format(input_shape))
num_train_samples

x_train shape: (2527, 384, 512, 1)
input_shape (384, 512, 1)


2527

In [36]:
# Reshape the image for the Summary API.
img = np.reshape(x_train[0], (-1, input_shape[0], input_shape[1] , 1))

In [38]:
# Clear out any prior log data.
!rm -rf logs/train_data/

# Sets up a timestamped log directory.
logdir = "logs/train_data/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

with file_writer.as_default():
    # Don't forget to reshape.
    num_images = 2
    images = np.reshape(x_train[0:num_images], (-1, input_shape[0], input_shape[1], 1))
    images = images * 255
    tensorboard_title = "{} training data examples".format(num_images)
    tf.summary.image(tensorboard_title, images, max_outputs=25, step=0)
    
    #plt.imshow(np.squeeze(images[0]), cmap='gray')
    
%tensorboard --logdir logs/train_data

Reusing TensorBoard on port 6007 (pid 9209), started 3:58:32 ago. (Use '!kill 9209' to kill it.)